# Q1

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error



In [2]:
# Load dataset
df = pd.read_csv('Passengers.csv')

# Extract the number of passengers
data = df['#Passengers'].values


In [3]:
# Define the Universe of Discourse (UOD)
min_val = np.min(data)
max_val = np.max(data)
uod_min = min_val * 0.9  # 10% decrement
uod_max = max_val * 1.1  # 10% increment

#Split UOD into equal-length intervals
interval_length = 10
intervals = np.arange(uod_min, uod_max + interval_length, interval_length)




In [4]:
#Fuzzify the time series data
def fuzzify(observation, intervals):
    for i in range(len(intervals) - 1):
        if intervals[i] <= observation < intervals[i + 1]:
            return i
    return len(intervals) - 2  # Handle edge case for max value



In [5]:
fuzzified_data = np.array([fuzzify(obs, intervals) for obs in data])

# Step 4: Create High Order Fuzzy Logical Relationships (FLRs)
order = 12  # Set order of Fuzzy TSF
X, y = [], []
for i in range(order, len(fuzzified_data)):
    X.append(fuzzified_data[i-order:i])
    y.append(fuzzified_data[i])
X = np.array(X)
y = np.array(y)



In [6]:
# Step 5: Split the data into Train, Validation, and Test sets (70-15-15 ratio)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, shuffle=False)

# Step 6: Train Linear Regression model on Train set
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)



LinearRegression()

In [7]:
# Step 7: Validate the model on Validation set
y_val_pred = lr_model.predict(X_val)
val_mse = mean_squared_error(y_val, y_val_pred)
print(f'Validation MSE: {val_mse}')



Validation MSE: 7.889446985827282


In [8]:
# Step 8: Test the model on Test set
y_test_pred = lr_model.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
print(f'Test MSE: {test_mse}')



Test MSE: 3.189577988577594


In [9]:
# Step 9: Defuzzify the results
def defuzzify(index, intervals):
    return (intervals[index] + intervals[index + 1]) / 2

# Apply defuzzification to the predicted results
y_test_defuzzified = np.array([defuzzify(int(np.round(pred)), intervals) for pred in y_test_pred])



In [10]:
# Step 10: Evaluate the model's performance
final_test_mse = mean_squared_error(data[len(data)-len(y_test):], y_test_defuzzified)


# Function to calculate RMSE
def calc_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))
final_test_rmse = calc_rmse(data[len(data)-len(y_test):], y_test_defuzzified)
print(f'Final Test MSE (Defuzzified): {final_test_mse}')
print(f'Final Test RMSE: {final_test_rmse}')



Final Test MSE (Defuzzified): 267.86000000000024
Final Test RMSE: 16.366429054622767


# Q2

In [11]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_csv('Passengers.csv')
data = df['#Passengers'].values

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data.reshape(-1, 1))




In [12]:
# Function to create sequences from the time series data
def create_sequences(data, time_step):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

# Set the time step (sequence length)
time_step = 12  # Order of fuzzy TSF model is 12



In [13]:
# Create sequences
X, y = create_sequences(data_scaled, time_step)

# Split into Train (70%), Validation (15%), Test (15%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, shuffle=False)

# Reshape the data for LSTM [samples, time steps, features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))



In [14]:
# Function to build the LSTM model
def create_lstm_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(time_step, 1)))  # Input layer
    model.add(tf.keras.layers.LSTM(64, return_sequences=False))  # LSTM layer with 64 units
    model.add(tf.keras.layers.Dense(32, activation='relu'))  # Fully Connected layer with 32 neurons
    model.add(tf.keras.layers.Dense(1))  # Output layer with single neuron
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Function to calculate RMSE
def calc_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))



In [15]:
# Repeat simulations for 10 times and calculate mean RMSE
n_simulations = 10
rmse_list = []



In [16]:
for i in range(n_simulations):
    print(f'Simulation {i+1}')
    # Build and train the model
    model = create_lstm_model()
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, verbose=0, batch_size=32)
    
    # Predict on test set
    y_test_pred = model.predict(X_test)
    
    # Inverse transform the predictions and actual test values to original scale
    y_test_pred = scaler.inverse_transform(y_test_pred)
    y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))
    
    # Calculate RMSE
    rmse = calc_rmse(y_test_actual, y_test_pred)
    rmse_list.append(rmse)
    print(f'RMSE for Simulation {i+1}: {rmse}')



Simulation 1


C:\Users\rksah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
RMSE for Simulation 1: 45.75829359461133
Simulation 2


C:\Users\rksah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
RMSE for Simulation 2: 59.39832832628563
Simulation 3


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
RMSE for Simulation 8: 41.41683540187964
Simulation 9


C:\Users\rksah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
RMSE for Simulation 9: 60.2622256491586
Simulation 10


C:\Users\rksah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
RMSE for Simulation 10: 65.86346392940224


In [17]:
# Calculate mean RMSE over 10 simulations
mean_rmse = np.mean(rmse_list)
print(f'Mean RMSE over 10 simulations: {mean_rmse}')



Mean RMSE over 10 simulations: 49.90410388870171


# Q3

In [18]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_csv('Passengers.csv')
data = df['#Passengers'].values

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data.reshape(-1, 1))




In [19]:
df.head()

,Month,#Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121


In [20]:
# Function to create sequences from the time series data
def create_sequences(data, time_step):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

# Set the time step (sequence length)
time_step = 12  # Order of the time series


In [21]:

# Create sequences
X, y = create_sequences(data_scaled, time_step)

# Split into Train (70%), Validation (15%), Test (15%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, shuffle=False)

# Reshape the data for GRU [samples, time steps, features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))



In [22]:
# Function to build the GRU model
def create_gru_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(time_step, 1)))  # Input layer
    model.add(tf.keras.layers.GRU(64, return_sequences=False))  # GRU layer with 64 units
    model.add(tf.keras.layers.Dense(32, activation='relu'))  # Fully Connected layer with 32 neurons
    model.add(tf.keras.layers.Dense(1))  # Output layer with single neuron
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Function to calculate RMSE
def calc_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Repeat simulations for 10 times and calculate mean RMSE
n_simulations = 10
rmse_list = []



In [23]:
for i in range(n_simulations):
    print(f'Simulation {i+1}')
    # Build and train the model
    model = create_gru_model()
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, verbose=0, batch_size=32)
    
    # Predict on test set
    y_test_pred = model.predict(X_test)
    
    # Inverse transform the predictions and actual test values to original scale
    y_test_pred = scaler.inverse_transform(y_test_pred)
    y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))
    
    # Calculate RMSE
    rmse = calc_rmse(y_test_actual, y_test_pred)
    rmse_list.append(rmse)
    print(f'RMSE for Simulation {i+1}: {rmse}')



Simulation 1


C:\Users\rksah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
RMSE for Simulation 1: 41.07125729918286
Simulation 2


C:\Users\rksah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
RMSE for Simulation 2: 45.943350933641604
Simulation 3


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
RMSE for Simulation 8: 42.24191940528657
Simulation 9


C:\Users\rksah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
RMSE for Simulation 9: 43.392869677262574
Simulation 10


C:\Users\rksah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
RMSE for Simulation 10: 38.546398661622014


In [24]:
# Calculate mean RMSE over 10 simulations
mean_rmse = np.mean(rmse_list)
print(f'Mean RMSE over 10 simulations: {mean_rmse}')



Mean RMSE over 10 simulations: 42.877551875068754


# Q4

In [25]:
# Among the three approaches, Q1 is the best approach